In [206]:
import pandas as pd
import numpy as np
from sklearn.linear_model import LogisticRegression, SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.experimental import enable_iterative_imputer
from sklearn.impute import IterativeImputer

kernel_stats = pd.read_csv('titanic/train.csv')
kernel_stats

,PassengerId,Survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...,...
886,887,0,2,"Montvila, Rev. Juozas",male,27.0,0,0,211536,13.0000,NaN,S
887,888,1,1,"Graham, Miss. Margaret Edith",female,19.0,0,0,112053,30.0000,B42,S
888,889,0,3,"Johnston, Miss. Catherine Helen ""Carrie""",female,NaN,1,2,W./C. 6607,23.4500,NaN,S
889,890,1,1,"Behr, Mr. Karl Howell",male,26.0,0,0,111369,30.0000,C148,C


In [207]:
stats = kernel_stats.to_numpy()
stats
X = np.concatenate((np.reshape(stats[:,2],(len(stats),1)),stats[:,4:8],np.reshape(stats[:,9],(len(stats),1)),np.reshape(stats[:,11],(len(stats),1))), axis = 1)
y = stats[:,1]
y=y.astype('int')

In [208]:
X

array([[3, 'male', 22.0, ..., 0, 7.25, 'S'],
       [1, 'female', 38.0, ..., 0, 71.2833, 'C'],
       [3, 'female', 26.0, ..., 0, 7.925, 'S'],
       ...,
       [3, 'female', nan, ..., 2, 23.45, 'S'],
       [1, 'male', 26.0, ..., 0, 30.0, 'C'],
       [3, 'male', 32.0, ..., 0, 7.75, 'Q']], dtype=object)

In [209]:
np.reshape(stats[:,2],(1,len(stats))).shape

(1, 891)

In [210]:
for i in range(len(X[:,1])):
    if X[i,1] == "male":
        X[i,1] = 0
    else:
        X[i,1] = 1

for i in range(len(X[:,6])):
    if X[i,6] == "S":
        X[i,6] = 0
    elif X[i,6] == "C":
        X[i,6] = 1
    else:
        X[i,6] = 2

In [211]:
X

array([[3, 0, 22.0, ..., 0, 7.25, 0],
       [1, 1, 38.0, ..., 0, 71.2833, 1],
       [3, 1, 26.0, ..., 0, 7.925, 0],
       ...,
       [3, 1, nan, ..., 2, 23.45, 0],
       [1, 0, 26.0, ..., 0, 30.0, 1],
       [3, 0, 32.0, ..., 0, 7.75, 2]], dtype=object)

In [212]:
imp = IterativeImputer(max_iter=10000, random_state=0)
imp.fit(X)
X_imp = imp.transform(X)

In [213]:
scaler = StandardScaler()
X_norm = scaler.fit_transform(X_imp)

In [214]:
sgdr = LogisticRegression()
sgdr.fit(X_norm, y)

LogisticRegression()

In [215]:
type(y[0])

numpy.int32

In [216]:
stats = pd.read_csv('titanic/test.csv')
stats

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,892,3,"Kelly, Mr. James",male,34.5,0,0,330911,7.8292,NaN,Q
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,363272,7.0000,NaN,S
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,240276,9.6875,NaN,Q
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,315154,8.6625,NaN,S
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,3101298,12.2875,NaN,S
...,...,...,...,...,...,...,...,...,...,...,...
413,1305,3,"Spector, Mr. Woolf",male,NaN,0,0,A.5. 3236,8.0500,NaN,S
414,1306,1,"Oliva y Ocana, Dona. Fermina",female,39.0,0,0,PC 17758,108.9000,C105,C
415,1307,3,"Saether, Mr. Simon Sivertsen",male,38.5,0,0,SOTON/O.Q. 3101262,7.2500,NaN,S
416,1308,3,"Ware, Mr. Frederick",male,NaN,0,0,359309,8.0500,NaN,S


In [217]:
stats = stats.to_numpy()
prompt = np.concatenate((np.reshape(stats[:,1],(len(stats),1)),stats[:,3:7],np.reshape(stats[:,8],(len(stats),1)),np.reshape(stats[:,10],(len(stats),1))), axis = 1)

In [218]:
prompt

array([[3, 'male', 34.5, ..., 0, 7.8292, 'Q'],
       [3, 'female', 47.0, ..., 0, 7.0, 'S'],
       [2, 'male', 62.0, ..., 0, 9.6875, 'Q'],
       ...,
       [3, 'male', 38.5, ..., 0, 7.25, 'S'],
       [3, 'male', nan, ..., 0, 8.05, 'S'],
       [3, 'male', nan, ..., 1, 22.3583, 'C']], dtype=object)

In [219]:
X = prompt
for i in range(len(X[:,1])):
    if X[i,1] == "male":
        X[i,1] = 0
    else:
        X[i,1] = 1

for i in range(len(X[:,6])):
    if X[i,6] == "S":
        X[i,6] = 0
    elif X[i,6] == "C":
        X[i,6] = 1
    else:
        X[i,6] = 2
X

array([[3, 0, 34.5, ..., 0, 7.8292, 2],
       [3, 1, 47.0, ..., 0, 7.0, 0],
       [2, 0, 62.0, ..., 0, 9.6875, 2],
       ...,
       [3, 0, 38.5, ..., 0, 7.25, 0],
       [3, 0, nan, ..., 0, 8.05, 0],
       [3, 0, nan, ..., 1, 22.3583, 1]], dtype=object)

In [220]:
results = np.concatenate((np.reshape(stats[:,0],(len(stats),1)),np.zeros((418,1))), axis=1)
results

array([[892, 0.0],
       [893, 0.0],
       [894, 0.0],
       [895, 0.0],
       [896, 0.0],
       [897, 0.0],
       [898, 0.0],
       [899, 0.0],
       [900, 0.0],
       [901, 0.0],
       [902, 0.0],
       [903, 0.0],
       [904, 0.0],
       [905, 0.0],
       [906, 0.0],
       [907, 0.0],
       [908, 0.0],
       [909, 0.0],
       [910, 0.0],
       [911, 0.0],
       [912, 0.0],
       [913, 0.0],
       [914, 0.0],
       [915, 0.0],
       [916, 0.0],
       [917, 0.0],
       [918, 0.0],
       [919, 0.0],
       [920, 0.0],
       [921, 0.0],
       [922, 0.0],
       [923, 0.0],
       [924, 0.0],
       [925, 0.0],
       [926, 0.0],
       [927, 0.0],
       [928, 0.0],
       [929, 0.0],
       [930, 0.0],
       [931, 0.0],
       [932, 0.0],
       [933, 0.0],
       [934, 0.0],
       [935, 0.0],
       [936, 0.0],
       [937, 0.0],
       [938, 0.0],
       [939, 0.0],
       [940, 0.0],
       [941, 0.0],
       [942, 0.0],
       [943, 0.0],
       [944,

In [221]:
imp = IterativeImputer(max_iter=10000, random_state=0)
imp.fit(X)
X_imp = imp.transform(X)

In [222]:
X_norm = scaler.fit_transform(X_imp)
X_norm

array([[ 0.87348191, -0.75592895,  0.38902184, ..., -0.4002477 ,
        -0.4971606 ,  2.24316522],
       [ 0.87348191,  1.32287566,  1.35699221, ..., -0.4002477 ,
        -0.51202396, -0.6778412 ],
       [-0.31581919, -0.75592895,  2.51855665, ..., -0.4002477 ,
        -0.46385067,  2.24316522],
       ...,
       [ 0.87348191, -0.75592895,  0.69877236, ..., -0.4002477 ,
        -0.50754273, -0.6778412 ],
       [ 0.87348191, -0.75592895, -0.35294054, ..., -0.4002477 ,
        -0.49320277, -0.6778412 ],
       [ 0.87348191, -0.75592895, -0.42052837, ...,  0.61989583,
        -0.23672728,  0.78266201]])

In [223]:
y_pred = sgdr.predict(X_norm)

In [224]:
y_pred

array([0, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 1, 1, 0, 0, 0, 0, 0, 0,
       1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 1,
       1, 0, 1, 1, 1, 0, 1, 1, 1, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1,
       1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0,
       0, 1, 1, 1, 1, 0, 0, 1, 1, 1, 1, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0,
       1, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0,
       0, 0, 1, 1, 0, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 0, 1,
       1, 0, 1, 1, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1,
       0, 1, 1, 0, 0, 1, 0, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 0, 1, 0,
       1, 0, 1, 0, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1,
       0, 0, 0, 0, 1, 0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 1, 0, 0, 0, 1, 1,
       0, 0, 0, 0, 1, 0, 0, 0, 1, 1, 0, 1, 0, 0, 0, 0, 1, 0, 1, 1, 1, 0,
       0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 0,

In [226]:
y_pred.shape

(418,)

In [229]:
results = np.concatenate((np.reshape(stats[:,0],(len(stats),1)),np.reshape(y_pred,(len(stats),1))), axis=1)

In [230]:
results

array([[892, 0],
       [893, 0],
       [894, 0],
       [895, 0],
       [896, 1],
       [897, 0],
       [898, 1],
       [899, 0],
       [900, 1],
       [901, 0],
       [902, 0],
       [903, 0],
       [904, 1],
       [905, 0],
       [906, 1],
       [907, 1],
       [908, 0],
       [909, 0],
       [910, 0],
       [911, 0],
       [912, 0],
       [913, 0],
       [914, 1],
       [915, 1],
       [916, 1],
       [917, 0],
       [918, 1],
       [919, 0],
       [920, 0],
       [921, 0],
       [922, 0],
       [923, 0],
       [924, 0],
       [925, 1],
       [926, 0],
       [927, 0],
       [928, 1],
       [929, 1],
       [930, 0],
       [931, 0],
       [932, 0],
       [933, 0],
       [934, 0],
       [935, 1],
       [936, 1],
       [937, 0],
       [938, 0],
       [939, 0],
       [940, 1],
       [941, 0],
       [942, 0],
       [943, 0],
       [944, 1],
       [945, 1],
       [946, 0],
       [947, 0],
       [948, 0],
       [949, 0],
       [950, 0

In [231]:
df = pd.DataFrame(results, columns=['PassengerId','Survived'])

In [232]:
df

,PassengerId,Survived
0,892,0
1,893,0
2,894,0
3,895,0
4,896,1
...,...,...
413,1305,0
414,1306,1
415,1307,0
416,1308,0


In [233]:
df.to_csv('results.csv')